In [1]:
import numpy as np
import copy
import networkx as nx
import networkx.algorithms.community as nx_comm

import matplotlib.pyplot as plt 
from   matplotlib import cm
from   matplotlib.ticker import LinearLocator, FormatStrFormatter
%config InlineBackend.figure_format = 'svg'

In [1]:
def compute_modularity(n_nodes, B, bitstring, C=None): #returns unnormalized modularity
    if isinstance(n_nodes, nx.Graph) or isinstance(n_nodes, nx.DiGraph):
        # legacy
        n_nodes = n_nodes.number_of_nodes()
    if 0 in bitstring:
        # assuming bitstring is of zeros and ones
        bitstring = [
            -1 if x == 0 else 1 if x == 1 else 'Error' for x in bitstring
        ]
    if 'Error' in bitstring or len(bitstring) != n_nodes:
        raise ValueError(
            "Incorrect bistring encountered. Only accepts bitstrings containing 0s and 1s or -1s and 1s of the size equal to number of nodes in G"
        )
    if not isinstance(bitstring, np.ndarray):
        bitstring = np.asarray(bitstring)
    if not isinstance(C, np.ndarray) and C is not None:
        C = np.asarray(C)
    cost = (bitstring.dot(B)).dot(bitstring.T)
    if C is not None:
        cost += C.T.dot(bitstring)
    
    return float(cost)


def compute_gain(G, B, curr_bitstring, v, return_v=False, graph2subgraph=None):
    reassigned = copy.deepcopy(curr_bitstring)
    #print(list(G.nodes()))
    #print(graph2subgraph)
    
    if graph2subgraph is None:
        reassigned[v] = -curr_bitstring[v]
    else:
        reassigned[graph2subgraph[v]] = -curr_bitstring[graph2subgraph[v]]
   
    
    gain = (
        compute_modularity(G.number_of_nodes(), B, reassigned) -
        compute_modularity(G.number_of_nodes(), B, curr_bitstring))
    
    if return_v:
        return gain, v
    else:
        return gain
